In [20]:
from tqdm import tqdm
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import os
from PIL import Image

In [2]:
# Define the transformation to be applied to each image
transform = transforms.Compose(
    [transforms.Resize((224, 224)),  # Resize the image to 224x224
     transforms.ToTensor(),  # Convert the image to a PyTorch tensor
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])  # Normalize the image


In [6]:
df = pd.read_csv('<path to evaluation csv file>')

In [7]:
dataset_resnet = torchvision.datasets.ImageFolder(root='<path to resnet dataset folder>', transform=transform)

In [8]:
resnet_model = torchvision.models.resnet18(pretrained=False)
num_ftrs = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(num_ftrs, len(dataset_resnet.classes))


/ext3/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/ext3/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [13]:

resnet_model.load_state_dict(torch.load('<path to saved trained resnet weigths eg:resnet1876.17260787992495.pth>'))

<All keys matched successfully>

In [10]:
images = df['images'].tolist()
class_list = df['classes'].tolist()
correct = 0
wrong = []
counter = 0


In [11]:
pokemon_a = '<path to pokemon-a dataset>'
pokemon_b = '<path to pokemon-b dataset>'

In [17]:
pokedex = pd.read_csv('<path to pokedex.csv>')

In [29]:
a_images = os.listdir(pokemon_a)

for img in a_images:
    img_path = pokemon_a + img
    index = img.split('.')[0]
    if "-" in index:
        index = index.split('-')[0]
    correct_label = list(pokedex[pokedex['Name'] == int(index)]['Type 1'])[0]
    
    image = Image.open(img_path)
    image = image.convert('RGB')
    image =  transform(image)
    image =  image.view(1, 3, 224, 224)
    resnet_model.eval()
    with torch.no_grad():
        predicted_resnet_label = resnet_model(image)
        predicted_resnet_label = torch.argmax(predicted_resnet_label)
        predicted_class = dataset_resnet.classes[predicted_resnet_label]
    if correct_label.lower() == predicted_class.lower():
        correct += 1
    else:
        print(predicted_class, correct_label, img)


Farfetchd Farfetch'd 83.png
Charmander Eevee 133.png
MrMime Mr. Mime 122.png
Nidorino Nidoran♂ 32.png
Mewtwo Mew 151.png
Nidorina Nidoran♀ 29.png


In [31]:
print((correct)/len(a_images))

0.9883040935672515


In [33]:
b_images = os.listdir(pokemon_b)

for img in b_images:
    img_path = pokemon_b + img
    index = img.split('.')[0]
    if "-" in index:
        index = index.split('-')[0]
    correct_label = list(pokedex[pokedex['Name'] == int(index)]['Type 1'])[0]
    
    image = Image.open(img_path)
    image = image.convert('RGB')
    image =  transform(image)
    image =  image.view(1, 3, 224, 224)
    resnet_model.eval()
    with torch.no_grad():
        predicted_resnet_label = resnet_model(image)
        predicted_resnet_label = torch.argmax(predicted_resnet_label)
        predicted_class = dataset_resnet.classes[predicted_resnet_label]
    if correct_label.lower() == predicted_class.lower():
        correct += 1
    else:
        print(predicted_class, correct_label, img)


Pidgeot Pidgeotto 17.jpg
Weepinbell Victreebel 71.jpg
Muk Grimer 88.jpg
Nidorino Nidoran♂ 32.jpg
Farfetchd Farfetch'd 83.jpg
MrMime Mr. Mime 122.jpg
Bulbasaur Ivysaur 2.jpg
Pidgeotto Pidgey 16.jpg
Grimer Muk 89.jpg
Mankey Primeape 57.jpg
Nidorina Nidoran♀ 29.jpg
Rhydon Charizard 6-mega-x.jpg
1.894736842105263


In [36]:
print((correct)/(len(b_images)+len(a_images)))

0.9707602339181286
